# Detección de objetos

## Instalación de librerías

In [1]:
!pip install opencv-python
!pip install matplotlib
!pip install numpy

## YoloV3

https://github.com/ultralytics/yolov3

In [2]:
!wget https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg

"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [3]:
!wget https://pjreddie.com/media/files/yolov3.weights

"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [4]:
!wget https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names

"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


## Deteccion en cámara web

- Cargamos librerías

In [5]:
import cv2
import numpy as np

- Cargamos la red neuronal de YOLOv3

In [6]:
# Cargar la red neuronal pre-entrenada YOLOv3 y sus configuraciones
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

- Función detección de objetos

In [7]:
# Función para detección de objetos
def detect_objects(img):
    height, width, channels = img.shape

    # Preprocesamiento de la imagen
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Información de detección
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Objeto detectado
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                # Coordenadas de la caja
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Supresión no máxima para eliminar cajas superpuestas
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Dibujar cajas y etiquetas en la imagen
    font = cv2.FONT_HERSHEY_PLAIN
    colors = np.random.uniform(0, 255, size=(len(classes), 3))
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = colors[i]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y + 30), font, 2, color, 2)

    return img


- Captura de cámara

In [ ]:
# Captura de video desde la cámara web
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640) # Ancho
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480) # Alto
while True:
    # Leer el fotograma de la cámara
    ret, frame = cap.read()
    # Si no se puede leer el fotograma, salir del bucle
    if not ret:
        break

    # Detectar objetos en el fotograma
    detected_frame = detect_objects(frame)

    # Mostrar el fotograma con objetos detectados
    cv2.imshow('Object Detection', detected_frame)

    # Salir del bucle si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar la captura de video y cerrar las ventanas
cap.release()
cv2.destroyAllWindows()